In [46]:
import Pkg; Pkg.rm("GR")

    Updating `C:\Users\augus\.julia\environments\v1.10\Project.toml`
  [28b8d3ca] - GR v0.72.10
  No Changes to `C:\Users\augus\.julia\environments\v1.10\Manifest.toml`
        Info We haven't cleaned this depot up for a bit, running Pkg.gc()...
      Active manifest files: 5 found
      Active artifact files: 129 found
      Active scratchspaces: 6 found
     Deleted no artifacts, repos, packages or scratchspaces


In [45]:
import Pkg; Pkg.add("JLD")

   Resolving package versions...
   Installed H5Zblosc ── v0.1.2
   Installed Lz4_jll ─── v1.9.4+0
   Installed Blosc_jll ─ v1.21.5+0
   Installed Blosc ───── v0.7.3
   Installed JLD ─────── v0.13.4
   Installed FileIO ──── v1.16.2
   Installed HDF5 ────── v0.17.1
    Updating `C:\Users\augus\.julia\environments\v1.10\Project.toml`
  [4138dd39] + JLD v0.13.4
    Updating `C:\Users\augus\.julia\environments\v1.10\Manifest.toml`
  [a74b3585] + Blosc v0.7.3
  [5789e2e9] + FileIO v1.16.2
  [c8ec2601] + H5Zblosc v0.1.2
  [f67ccb44] + HDF5 v0.17.1
  [4138dd39] + JLD v0.13.4
  [0b7ba130] + Blosc_jll v1.21.5+0
  [5ced341a] + Lz4_jll v1.9.4+0
Precompiling project...
  ✓ Lz4_jll
  ✓ Blosc_jll
  ✓ FileIO
  ✓ Blosc
  ✓ HDF5
  ✓ Plots → FileIOExt
  ✓ H5Zblosc
  ✓ JLD
  8 dependencies successfully precompiled in 34 seconds. 220 already precompiled.


In [48]:
using Test, LinearAlgebra, PlutoUI, Plots, NCDatasets , JuMP , CPLEX, AmplNLWriter, Statistics, NCDatasets,  FilesystemDatastructures, JLD
include("extract_data_from_netcdf.jl")
include("extract_data_from_netcdf_hydro.jl")
include("solve_unit_commitment_hydro.jl")

solve_unit_commitment_hydro (generic function with 1 method)

In [5]:
v = readdir("C:/Users/augus/cours-ensta/Opt discrete/T-Ramp") # obtenir un vecteur de chaînes de caractères avec les noms des fichiers et des sous-dossiers du dossier C:/Users/Mak/Desktop
vf = v[1:length(v)]

42-element Vector{String}:
 "100_0_1_w.nc4"
 "100_0_2_w.nc4"
 "100_0_3_w.nc4"
 "100_0_4_w.nc4"
 "100_0_5_w.nc4"
 "10_0_1_w.nc4"
 "10_0_2_w.nc4"
 "10_0_3_w.nc4"
 "10_0_4_w.nc4"
 "10_0_5_w.nc4"
 "150_0_1_w.nc4"
 "150_0_2_w.nc4"
 "150_0_3_w.nc4"
 ⋮
 "20_0_5_w.nc4"
 "50_0_1_w.nc4"
 "50_0_2_w.nc4"
 "50_0_3_w.nc4"
 "50_0_4_w.nc4"
 "50_0_5_w.nc4"
 "75_0_1_w.nc4"
 "75_0_2_w.nc4"
 "75_0_3_w.nc4"
 "75_0_4_w.nc4"
 "75_0_5_w.nc4"
 "Aurland_1000.nc4"

In [11]:
#cor_mat  = Array{Float64}(undef,length(vf),8) 

42×8 Matrix{Float64}:
   0.0             0.0           …    0.0             0.0
   0.0             6.95217e-310       0.0             6.95217e-310
 NaN               6.95217e-310     NaN               6.95222e-310
   1.42491e-311    0.0                1.42491e-311    0.0
   0.0             0.0                1.42491e-311    0.0
   0.0           NaN             …    0.0           NaN
   5.4e-323        0.0                0.0             0.0
   1.42491e-311    0.0                6.95216e-310    0.0
   1.42387e-311    0.0                1.4249e-311     0.0
   1.42463e-311    0.0                1.42488e-311    0.0
   1.42387e-311    6.95217e-310  …    0.0             6.95217e-310
   0.0             6.95217e-310     NaN               6.95217e-310
   0.0             0.0                0.0             0.0
   ⋮                             ⋱                  
   0.0             0.0           …    0.0             0.0
   0.0             0.0                0.0             0.0
   0.0           NaN 

In [16]:
for i in 1:2
    file=vf[i]
    print(file,"   ",i)
    T, m, Δt, ThUnits, min_power, max_power, delta_ramp_up, delta_ramp_down, start_up_cost, linear_term, const_term, quad_term, d, InitUpDownTime, MinDownTime, MinUpTime, InitialPower = extract_data_from_netcdf("T-ramp/$file")
    StartArc, EndArc, Inflows, InitialVolume, MaxVolume, InitialFlowRate, DeltaRampUp_hydro, DeltaRampDown_hydro, MinFlow, MaxFlow, MinPower, MaxPower,LinearTerm_hydro,ConstantTerm_hydro =extract_data_from_netcdf_hydro("T-ramp/Aurland_1000.nc4")
    solution_x, solution_p, solution_cost, solution_xhydro, solution_phydro, solution_f = solve_unit_commitment_hydro(InitialPower,T, m, Δt, ThUnits, min_power, max_power, delta_ramp_up, delta_ramp_down, start_up_cost, linear_term, const_term, quad_term, d, InitUpDownTime,MinDownTime,MinUpTime,StartArc, EndArc, Inflows, InitialVolume, MaxVolume, InitialFlowRate, DeltaRampUp_hydro, DeltaRampDown_hydro, MinFlow, MaxFlow, MinPower, MaxPower,LinearTerm_hydro,ConstantTerm_hydro)
    L =[min_power, max_power, delta_ramp_up, delta_ramp_down, start_up_cost, linear_term, const_term, quad_term]
    thermal_power = sum(solution_p[i,:] for i in 1:length(ThUnits))/1000
    hydro_power = sum(solution_phydro[i,:] for i in 1:length(StartArc))/1000
    total = hydro_power+thermal_power

    p=plot([thermal_power hydro_power d/1000 total],label = ["Thermal Power" "Hydro Power" "Demand" "Total"])
    title!("Production  totale $file")
    xlabel!("h")
    ylabel!("Mw")
    savefig("\image\total_prod")
# plot([total])
    
end

150_0_1_w.nc4   11Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
Tried aggregator 2 times.
MIP Presolve eliminated 16986 rows and 8198 columns.
MIP Presolve modified 17685 coefficients.
Aggregator did 1239 substitutions.
Reduced MIP has 17856 rows, 5971 columns, and 51078 nonzeros.
Reduced MIP has 2457 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.12 sec. (96.80 ticks)
Found incumbent of value 2.7346691e+07 after 0.19 sec. (174.39 ticks)
Probing changed sense of 1687 constraints.
Probing time = 0.09 sec. (14.91 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 3164 rows and 1687 columns.
MIP Presolve modified 4149 coefficients.
Reduced MIP has 14692 rows, 4284 columns, and 41094 nonzeros.
Reduced MIP has 770 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (32.07 ticks)
Probing time = 0.01 sec. (3.92 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 14692 rows, 4284 columns, and 41094 nonzeros.
Reduced MIP 

150_0_4_w.nc4   14Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
Tried aggregator 2 times.
MIP Presolve eliminated 16986 rows and 8198 columns.
MIP Presolve modified 17700 coefficients.
Aggregator did 1236 substitutions.
Reduced MIP has 17862 rows, 5974 columns, and 51212 nonzeros.
Reduced MIP has 2460 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.14 sec. (96.92 ticks)
Found incumbent of value 2.3983722e+07 after 0.22 sec. (145.01 ticks)
Probing changed sense of 1523 constraints.
Probing time = 0.11 sec. (15.03 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 2836 rows and 1501 columns.
MIP Presolve modified 3836 coefficients.
Aggregator did 22 substitutions.
Reduced MIP has 15004 rows, 4451 columns, and 42188 nonzeros.
Reduced MIP has 937 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.09 sec. (58.65 ticks)
Probing time = 0.03 sec. (4.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 15004 rows, 4451 columns

200_0_10_w.nc4   16Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
Tried aggregator 2 times.
MIP Presolve eliminated 23786 rows and 10648 columns.
MIP Presolve modified 13388 coefficients.
Aggregator did 1336 substitutions.
Reduced MIP has 22462 rows, 8224 columns, and 63067 nonzeros.
Reduced MIP has 3560 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.16 sec. (113.86 ticks)
Found incumbent of value 3.8446547e+07 after 0.36 sec. (315.91 ticks)
Probing changed sense of 2418 constraints.
Probing time = 0.17 sec. (19.73 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 2991 rows and 2384 columns.
MIP Presolve modified 5490 coefficients.
Aggregator did 34 substitutions.
Reduced MIP has 19437 rows, 5806 columns, and 54317 nonzeros.
Reduced MIP has 1142 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.11 sec. (74.82 ticks)
Probing time = 0.02 sec. (4.40 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 19437 rows, 5806 col

200_0_1_w.nc4   18Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
Tried aggregator 2 times.
MIP Presolve eliminated 21886 rows and 10648 columns.
MIP Presolve modified 23631 coefficients.
Aggregator did 1605 substitutions.
Reduced MIP has 23824 rows, 7955 columns, and 68232 nonzeros.
Reduced MIP has 3291 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.19 sec. (128.38 ticks)
Found incumbent of value 3.6298570e+07 after 0.31 sec. (245.88 ticks)
Probing changed sense of 2292 constraints.
Probing time = 0.14 sec. (19.80 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 4300 rows and 2292 columns.
MIP Presolve modified 5592 coefficients.
Reduced MIP has 19524 rows, 5663 columns, and 54595 nonzeros.
Reduced MIP has 999 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.06 sec. (42.63 ticks)
Probing time = 0.03 sec. (4.42 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 19524 rows, 5663 columns, and 54595 nonzeros.
Reduced MI

200_0_3_w.nc4   20Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
Tried aggregator 2 times.
MIP Presolve eliminated 21886 rows and 10648 columns.
MIP Presolve modified 23623 coefficients.
Aggregator did 1608 substitutions.
Reduced MIP has 23818 rows, 7952 columns, and 68200 nonzeros.
Reduced MIP has 3288 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.17 sec. (128.35 ticks)
Found incumbent of value 3.2883623e+07 after 0.27 sec. (217.43 ticks)
Probing changed sense of 2148 constraints.
Probing time = 0.12 sec. (19.79 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 4030 rows and 2148 columns.
MIP Presolve modified 5368 coefficients.
Reduced MIP has 19788 rows, 5804 columns, and 55548 nonzeros.
Reduced MIP has 1140 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.06 sec. (43.25 ticks)
Probing time = 0.03 sec. (4.45 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 19788 rows, 5804 columns, and 55548 nonzeros.
Reduced M

...

20_0_3_w.nc4   29Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
Tried aggregator 2 times.
MIP Presolve eliminated 4246 rows and 1828 columns.
MIP Presolve modified 2376 coefficients.
Aggregator did 239 substitutions.
Reduced MIP has 2436 rows, 861 columns, and 6950 nonzeros.
Reduced MIP has 337 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (14.47 ticks)
Found incumbent of value 2280716.038180 after 0.03 sec. (18.45 ticks)
Probing changed sense of 234 constraints.
Probing time = 0.03 sec. (7.33 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 440 rows and 234 columns.
MIP Presolve modified 614 coefficients.
Reduced MIP has 1996 rows, 627 columns, and 5621 nonzeros.
Reduced MIP has 103 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (4.60 ticks)
Probing time = 0.00 sec. (1.88 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve modified 1 coefficients.
Reduced MIP has 1996 rows, 627 columns, and 5621 nonz

50_0_2_w.nc4   33Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
Tried aggregator 2 times.
MIP Presolve eliminated 7186 rows and 3298 columns.
MIP Presolve modified 5904 coefficients.
Aggregator did 478 substitutions.
Reduced MIP has 5978 rows, 2032 columns, and 17187 nonzeros.
Reduced MIP has 818 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.06 sec. (33.93 ticks)
Found incumbent of value 7669336.735068 after 0.09 sec. (50.89 ticks)
Probing changed sense of 481 constraints.
Probing time = 0.03 sec. (4.88 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 904 rows and 481 columns.
MIP Presolve modified 1203 coefficients.
Reduced MIP has 5074 rows, 1551 columns, and 14423 nonzeros.
Reduced MIP has 337 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (11.23 ticks)
Probing time = 0.00 sec. (1.87 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 5074 rows, 1551 columns, and 14423 nonzeros.
Reduced MIP has 337 bina

50_0_4_w.nc4   35Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
Tried aggregator 2 times.
MIP Presolve eliminated 7661 rows and 3298 columns.
MIP Presolve modified 3354 coefficients.
Aggregator did 404 substitutions.
Reduced MIP has 5651 rows, 2106 columns, and 15885 nonzeros.
Reduced MIP has 892 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.05 sec. (30.17 ticks)
Found incumbent of value 7304273.796562 after 0.06 sec. (45.18 ticks)
Probing changed sense of 541 constraints.
Probing time = 0.05 sec. (5.12 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 697 rows and 541 columns.
MIP Presolve modified 1216 coefficients.
Reduced MIP has 4954 rows, 1565 columns, and 13809 nonzeros.
Reduced MIP has 351 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (10.63 ticks)
Probing time = 0.00 sec. (1.90 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 4954 rows, 1565 columns, and 13809 nonzeros.
Reduced MIP has 351 bina

75_0_3_w.nc4   39Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
Tried aggregator 2 times.
MIP Presolve eliminated 9636 rows and 4523 columns.
MIP Presolve modified 8898 coefficients.
Aggregator did 655 substitutions.
Reduced MIP has 8974 rows, 3030 columns, and 25758 nonzeros.
Reduced MIP has 1241 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.08 sec. (49.57 ticks)
Found incumbent of value 1.2462034e+07 after 0.11 sec. (72.17 ticks)
Probing changed sense of 768 constraints.
Probing time = 0.06 sec. (7.76 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 1442 rows and 768 columns.
MIP Presolve modified 1969 coefficients.
Reduced MIP has 7532 rows, 2262 columns, and 21217 nonzeros.
Reduced MIP has 473 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (16.57 ticks)
Probing time = 0.01 sec. (2.67 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 7532 rows, 2262 columns, and 21217 nonzeros.
Reduced MIP has 473 bin

LoadError: NetCDF error: [31mNetCDF: Invalid dimension ID or name[39m (NetCDF error code: -46)

In [29]:
cor_mat[42,:]

8-element Vector{Float64}:
   0.0
 NaN
   1.4249000986e-311
   0.0
 NaN
   6.95217064910836e-310
   0.0
 NaN

In [30]:
mean_cor =Array{Float64}(undef,1,8)  
for i in 1:8
    mean_cor[i]=mean(cor_mat[:41,i])
end

In [49]:
save("mean_cor.jld","mean_cor",mean_cor)

In [90]:
using Plots.PlotMeasures # charger le module 
Names =["min_power", "max_power", "ramp_up", "ramp_down", "start_up_cost" ,"linear_term" ,"const_term"]
p=Plots.plot(Names,mean_cor'[1:7],seriestype=:scatter,label ="Correlation",size=(700,300), margin = 13px)
title!("Corrélation ")
xlabel!("Paramètre")
ylabel!("rho")   

Plots.savefig(p,"correlation.png")


"C:\\Users\\augus\\cours-ensta\\Opt discrete\\correlation.png"